In [32]:
import plotly.graph_objects as go
import requests
import datetime
from datetime import datetime
import string
import random
import calendar
import time
import sklearn
from matplotlib.lines import Line2D
import statsmodels.tsa.stattools as ts
import matplotlib.ticker as mtick
import matplotlib.collections as collections
from numpy import linalg
import os
import sys
import pandas as pd 
import numpy as np
import warnings
from sklearn import datasets
warnings.filterwarnings("ignore")
import math
import operator
import statsmodels.api as sm
import statsmodels.tsa.stattools as ts
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import matplotlib.font_manager as fm
from sklearn import preprocessing
from sklearn import linear_model
from statsmodels.tsa.arima_model import ARIMA
from sklearn.decomposition import PCA
from scipy.stats import pearsonr
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA
from scipy import stats
from copy import deepcopy
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from datetime import datetime
from WindPy import w
w.start()
from sklearn.tree import DecisionTreeRegressor 
from sklearn.ensemble import RandomForestRegressor 
from sklearn.feature_extraction import DictVectorizer
import csv
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import svm
from sklearn.model_selection import cross_validate
#趋势划分函数

def und(data, l1, l2):
    i = 0
    res = [0] * len(data)

    while i < len(data)-1:
        if i + l1 >= len(data) - 1:
            for j in range(i, len(data)):
                res[j] = res[i]
            break
        cur = data[i]
        
        next_id = min([(data).index(max(data[i+1+l1:i+1+l2])), data.index(min(data[i+1+l1:i+1+l2]))])
        res[i:next_id+1] = [- (-1) ** (data[next_id] > data[i])] * (next_id + 1 - i)
        i = next_id
    return res

#移动平均线MA
def ma(data, l_ma):
    res = []
    for i in range(l_ma-1,len(data)):
        res.append(np.mean(data[i-l_ma+1:i+1]))
    return res

#ROCSMA变化率移动平均
def rocsma(d1,i1,l_roc,l_sma):
    res = 0
    for x in range(l_sma):
        res += (d1[i1-x-1]/d1[i1-x-l_sma-1]-1)/l_sma
    return res

#KST公式
def kst(d2,i2,roc_l1,roc_l2,roc_l3,roc_l4,sma_l1,sma_l2,sma_l3,sma_l4):
    rocma1 = rocsma(d2,i2,roc_l1,sma_l1)
    rocma2 = rocsma(d2,i2,roc_l2,sma_l2)
    rocma3 = rocsma(d2,i2,roc_l3,sma_l3)
    rocma4 = rocsma(d2,i2,roc_l4,sma_l4)
    return (rocma1 * 1 + rocma2 * 2 + rocma3 * 3 + rocma4 * 4)
    

#相对强度
def rel_str(data, i1, i2):
    return (np.array(data[i1])/np.mean(data[i1])) / (np.array(data[i2])/np.mean(data[i2]))

#交叉变换趋势
def cross(d1,d2,cd):
    comp = [-(-1) ** (d1[i] > d2[i]) for i in range(len(d1))]
    res = [-comp[0]]
    c = 0
    
    for i in range(1,len(comp)):
        if comp[i] * comp[i-1] == -1 and c == 0:
            sl1 = -(-1)**(d1[i]>d1[i-1])
            sl2 = -(-1)**(d2[i]>d2[i-1])
            res.append(sl1*sl2)
            c += cd
        else:
            res.append(res[-1])
            c = max(0,c-1)
    return res

#大小对比
def comp(d1,d2):
    res = [-(-1) ** (d1[i] > d2[i]) for i in range(len(d1))]
    return res

#标准化
def std(data):
    res = []
    for i in range(len(data)):
        res.append((data[i] - np.mean(data[:i+1]))/np.mean(data[:i+1]))
    return res

#普林格经济周期
def pring(b,s,c):
    if b == 1 and s == c == -1:
        return -3
    if b == s == 1 and c == -1:
        return -1
    if b == s == c == 1:
        return 1
    if b == -1 and s == c == 1:
        return 3
    if b == s == -1 and c == 1:
        return 1
    if b == s == c == -1:
        return -1
    else:
        return 0

def dupe_legend(label, color):
    line = Line2D([0], [0], linestyle='none', mfc= 'white',
                mec=color, marker=r'$\mathregular{{{}}}$'.format(label))
    return line

#平滑折线
def smooth(data):
    for i in range(len(data)-4):
        if data[i] * data[i+2] == 1 and data[i] * data[i+1] == -1:
            data[i+1] = data[i]
        elif data[i] * data[i+3] == 1 and all(data[i+1:i+3]) != data[i]:
            for j in range(2):
                data[i+j+1] = data[i]
        elif data[i] * data[i+4] == 1 and all(data[i+1:i+4]) != data[i]:
            for j in range(3):
                data[i+j+1] = data[i]
    return data

def smooth2(data):
    for i in range(len(data)-5):
        if data[i] * data[i+5] == 1 and all(data[i+1:i+4]) != data[i]:
            for j in range(4):
                data[i+j+1] = data[i]
    return data
    
######################################评估方式

#准确度计算
def acc(d1,d2):
    return round(sum(np.array(d1) == np.array(d2))/ len(d1),3)

#收益率计算
def profit(data, ud):
    money = 1
    num = 0
    prt = 0   #显示路径
    for i in range(len(data)):
        if ud[i] == 1 and num == 0:
            if prt:
                print('in', date[11:][i], 'money', money, 'num', num, 'price', data[i])
            num = money/data[i]
            money = 0
        if ud[i] != 1 and money == 0:
            if prt:
                print('out', date[11:][i], 'money', money, 'num', num, 'price', data[i])
            money = num * data[i]
            num = 0
    if prt:
        print(max(money,num*data[-1]))
    return max(money,num*data[-1])

#复杂度计算
def cpx(x):
    res = 1
    dirc = x[0]
    for i in range(len(x)):
        if x[i] != dirc:
            res += 1
            dirc = x[i]
    return res/len(x)

In [10]:
int_list = [['M0020188','股票'],['S0105896','商品'],['M0265832','中债总净价指数'],['G1000116','OECD'],['M0043815','短贷利率']]

#起止时间
t0 = '2004-06-01'
t1 = '2020-10-01'

#指数取值周期
dic = {'D':'天', 'W':'周', 'M':'月', 'Q':'季度', 'S':'半年', 'Y':'年'}
prd = 'M'

#移动平均线平均时长
l = 12

ww = w.wnq("600519.SH")
ww

.ErrorCode=0
.Codes=[600519.SH]
.Fields=[id,title,time,url,source,abstract,relevant_windcodes,important]
.Times=[20201022 17:40:04]
.Data=[[513390717],[每一针都飘着茅台味道的玻尿酸，为何如此暴利？],[2020/10/22 17:40:04],[http://news.windin.com/ns/findsnap.php?code=89D4C251144A&sourcename=news.windin.com&ad=0&sitetype=0&sourcetype=0&id=513390717],[广发证券],[None],[600519.SH,300896.SZ,688363.SH,A12288.SZ,688366.SH,SY.O,399324.SZ],[0]]

In [23]:
target = ww.Data[3][0]
req = requests.get(url=target)
txt = req.text

In [33]:
ind_1 = 0
res = ''
for i in range(len(txt)):
    if txt not in string.punctuation:
        if txt[i] == '<':
            ind_1 = 1
        if ind == 0:
            res += txt[i]
        if txt[i] == '>':
            ind_1 = 0
res

'每一针都飘着茅台味道的玻尿酸，为何如此暴利？\r\n        var __server_show_flag=\'[show-flag]\';\r\n        function doWMShare() {\r\n            var escapeTitle = "每一针都飘着茅台味道的玻尿酸，为何如此暴利？";\r\n            window.external.ClientFunc("{func:\'command\', isGlobal:\'1\', cmdid:\'149\', OPType:\'8\', NewsTitle:\'" + escapeTitle + "\', NewsUrl:\'https://snap.windin.com/ns/findsnap.php?ad=0&sitetype=0&sourcetype=0&newsId=513390717&t=null&share=wm\', NewsID:\'513390717\', Time:\'2020-10-22 17:40:04 \', SourceType:\'0\'}");\r\n        }\r\n      每一针都飘着茅台味道的玻尿酸，为何如此暴利？2020-10-22 17:40来源：广发证券扫一扫,用手机看新闻！用微信扫描还可以分享至好友和朋友圈！“颜值经济”火爆的今天，火了各种医美神器，玻尿酸、水光针、肉毒素等，当然，也富了背后的公司。 十一前夕，“医美龙头股”爱美客正式在深交所上市，发行价为118.27元，成为创业板注册制以来发行价最高的股票。上市首日，爱美客涨幅一度涨超200%，股价最高达到366元以上。截至10月9日，爱美客市值已轻松破400亿。 爱美客的成功上市，也让其背后的创始人、57岁的女老板简军，成为新晋的 “玻尿酸女王”，其最新身家已经高达200亿左右。  简军的成功，也让不少人想到了同样以玻尿酸业务快速起家的华熙生物的女董事长赵燕，凭借华熙生物的快速扩张，赵燕已跃升至山东女首富、且荣登福布斯排行榜。  随着华熙生物、昊海生科、爱美客等玻尿酸三巨头先后上市，玻尿酸产业也逐渐揭开了神秘面纱。它不仅仅因为可以让女人抵抗年龄的烦扰而备受女性关注，更因其疯狂的暴利模式受到了资本市场的高度关注。  根据公开的数据显示，玻尿酸的平均毛利